Tercer ejercicio de Spark Streaming usando el API DStream.
WordCount con ventana deslizante

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
# Function to create and setup a new StreamingContext
def functionToCreateContext():
    # Create a local StreamingContext with two working thread and batch interval of 5 seconds
    sc = SparkContext("local[2]", "WindowedNetworkWordCount")
    ssc = StreamingContext(sc, 5)
    
    # Mandatory set a checkpoint dir
    # http://spark.apache.org/docs/latest/streaming-programming-guide.html#checkpointing
    #Crear carpeta /checkpointDirectory dentro del directorio notebooks-spark o dentro del directorio Spark-Streaming-Python-Examples
    ssc.checkpoint("./checkpointDirectory")  # set checkpoint directory
    return ssc

In [ ]:
# Get StreamingContext from checkpoint data or create a new one
ssc = StreamingContext.getOrCreate(checkpointPath = "./checkpointDirectory", setupFunc = functionToCreateContext)

In [ ]:
# Create a DStream that will connect to hostname:port, like localhost:9999
# In command line -> nc -lk 9999
lines =ssc.socketTextStream("localhost", 9999)

In [ ]:
# Split each line into words
words =lines.flatMap(lambda line:line.split(" "))

In [ ]:
# Count each word in each batch
pairs = words.map(lambda word:(word, 1))

In [ ]:
# Reduce last 30 seconds of data, every 10 seconds
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x,y:x+y, lambda x,y:x-y, 30, 10)

In [ ]:
# Print the first ten elements of each RDD generated in this DStream to the console
windowedWordCounts.pprint()

ssc.start() # Start the computation
ssc.awaitTermination() # Wait for the computation to terminate